In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Data Mining/aspect_based

/content/drive/MyDrive/Data Mining/aspect_based


In [3]:
import os
import re
import requests
import unicodedata
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from warnings import filterwarnings
from sklearn.preprocessing import LabelEncoder

In [4]:
mpl.rcParams['axes.spines.left'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.bottom'] = False

In [5]:
tqdm.pandas()

In [ ]:
STOPWORDS_BAHASA = stopwords.words("indonesian") + [',', 'di', ":"]

In [ ]:
[i for i in os.listdir() if "label" in i]

['ABSA_data_bank_label.xlsx',
 'ABSA_data_pangan_label.xlsx',
 'ABSA_data_pemerintah_label.xlsx']

# 1. Feature Engineering

In [6]:
# read preprocessed dataset
data_bank_final   = pd.read_excel("final_data_bank.xlsx")
data_pangan_final = pd.read_excel("final_data_pangan.xlsx") 
data_gove_final   = pd.read_excel("final_data_pemerintah.xlsx")

## 1.1 Label Encoder of Sentiment and Aspect on Bank Dataset

In [ ]:
# define label encoder
encoder_sentiment = LabelEncoder()
encoder_aspect    = LabelEncoder()

In [ ]:
# transform label data into numerical
data_bank_final['aspect_numerical'] = encoder_aspect.fit_transform(data_bank_final['aspect'])
data_bank_final['label_numerical']  = encoder_sentiment.fit_transform(data_bank_final['label'])

In [ ]:
# overview dataset
data_bank_final.head()

,sentence,aspect,label,aspect_numerical,label_numerical
0,Jakarta ANTARA Sejumlah informasi penting men...,Economy,neutral,2,1
1,mulai dari turunnya cadangan devisa Indonesia ...,Economy,neutral,2,1
2,Presiden ingatkan BPJS Ketenagakerjaan kelola ...,Economy,neutral,2,1
3,Ketua IMF soroti perubahan mendasar dalam ekon...,Economy,neutral,2,1
4,Ketua IMF soroti perubahan mendasar dalam ekon...,Economy,neutral,2,1


## 1.2 Label Encoder of Sentiment and Aspect on Food Dataset

In [ ]:
# define label encoder
encoder_sentiment = LabelEncoder()
encoder_aspect    = LabelEncoder()

In [ ]:
# transform label data into numerical
data_pangan_final['aspect_numerical'] = encoder_aspect.fit_transform(data_pangan_final['aspect'])
data_pangan_final['label_numerical']  = encoder_sentiment.fit_transform(data_pangan_final['label'])

In [ ]:
# overview dataset
data_pangan_final.head()

,sentence,aspect,label,aspect_numerical,label_numerical
0,Jakarta ANTARA Sejumlah informasi penting men...,Economy,neutral,0,1
1,mulai dari turunnya cadangan devisa Indonesia ...,Economy,neutral,0,1
2,Presiden ingatkan BPJS Ketenagakerjaan kelola ...,Economy,neutral,0,1
3,Ketua IMF soroti perubahan mendasar dalam ekon...,Economy,neutral,0,1
4,Ketua IMF soroti perubahan mendasar dalam ekon...,Economy,neutral,0,1


## 1.3 Label Encoder of Sentiment and Aspect on Goverment Dataset

In [7]:
# define label encoder
encoder_sentiment = LabelEncoder()
encoder_aspect    = LabelEncoder()

In [8]:
# transform label data into numerical
data_gove_final['aspect_numerical'] = encoder_aspect.fit_transform(data_gove_final['aspect'])
data_gove_final['label_numerical']  = encoder_sentiment.fit_transform(data_gove_final['label'])

In [9]:
# overview dataset
data_gove_final.head()

,sentence,aspect,label,aspect_numerical,label_numerical
0,8 miliar dolar ASBank Indonesia BI mencatat po...,Banking,neutral,0,1
1,setelah laporan pekerjaan yang kuat untuk Sept...,Banking,neutral,0,1
2,Pasar uang meningkatkan peluang menjadi 92 per...,Banking,neutral,0,1
3,Presiden The Fed New York John Williams mengat...,Banking,neutral,0,1
4,Data tersebut memperkuat kenaikan suku bunga s...,Banking,neutral,0,1


# 2. Data Modeling

In [10]:
# !pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 7.7 MB/s 
     |████████████████████████████████| 163 kB 60.0 MB/s 
     |████████████████████████████████| 6.6 MB 49.9 MB/s 


In [11]:
import gc
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.layers import Input, Dropout, Dense 
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from transformers import TFBertModel, BertConfig, BertTokenizerFast

In [12]:
tf.config.run_functions_eagerly(True)

In [13]:
# Setup BERT
MODEL_NAME = "cahya/bert-base-indonesian-522M"
ACC_TOKEN  = "<YOUR_TOKEN>"
MAX_LENGTH = 64
config = BertConfig.from_pretrained(MODEL_NAME, use_auth_token = ACC_TOKEN)
config.output_hidden_states = False

Downloading:   0%|          | 0.00/468 [00:00<?, ?B/s]

In [14]:
# define build general model BERT
def build_model(bert, data, target_1, target_2):
    
    BERT_MODEL = bert.layers[0]
    
    input_ids = Input(shape = (MAX_LENGTH, ), name = "input_ids", dtype = "int32")
    attn_mask = Input(shape = (MAX_LENGTH, ), name = "attention_mask", dtype = "int32")
    inputs    = {'input_ids' : input_ids, "attention_mask" : attn_mask}
    
    pretrained_model = BERT_MODEL(inputs)[1]
    dropout   = Dropout(config.hidden_dropout_prob, name = "pool_out")
    pooled    = dropout(pretrained_model, training = False)
    
    sentiment = Dense(
        units = len(data[target_1].value_counts()), 
        kernel_initializer = TruncatedNormal(stddev = config.initializer_range),
        name = "Sentiment"
    )(pooled)
    aspect  = Dense(
        units = len(data[target_2].value_counts()), 
        kernel_initializer = TruncatedNormal(stddev = config.initializer_range),
        name = "Aspect"
    )(pooled)
    outputs = {"sentiment" : sentiment, "aspect" : aspect}
    
    model = Model(inputs = inputs, outputs = outputs, name = "BERT_Bank_Multiclass")
    
    return model

In [15]:
# define hyperparams
optimizer = Adam(learning_rate = 5e-05, epsilon = 1e-08, decay = 0.01, clipnorm = 1.0)
loss = {"sentiment" : CategoricalCrossentropy(from_logits = True), "aspect" : CategoricalCrossentropy(from_logits = True)}
metrics = {"sentiment" : CategoricalAccuracy('accuracy'), "aspect" : CategoricalAccuracy('accuracy')}

In [16]:
# define callbacks
class OwnCallbacks(Callback):
    def on_epoch_end(self, epoch, logs = {}):
        if logs.get("Aspect_accuracy") > 0.8:
            gc.collect()
            self.model.stop_training = True

## 2.1. Bank Dataset

In [ ]:
# split data into train and test
data_bank_final, data_bank_final_test = train_test_split(data_bank_final, test_size = 0.2, stratify = data_bank_final["aspect_numerical"])

# check dimension of data after splitted
print(data_bank_final.shape, data_bank_final_test.shape)

(27200, 5) (6800, 5)


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = MODEL_NAME, config = config)
model = TFBertModel.from_pretrained(MODEL_NAME, config)

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545M [00:00<?, ?B/s]

Some layers from the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# inherit model
model_bert_bank = build_model(model, data_bank_final, "label_numerical", "aspect_numerical")

In [ ]:
# overview model architecture
model_bert_bank.summary()

Model: "BERT_Bank_Multiclass"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 64)]         0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 64)]         0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  110617344   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]']              
                                tentions(last_hidde                                               
                                n_state=(None, 64,                             

In [ ]:
# compile model
model_bert_bank.compile(optimizer = optimizer, loss = loss, metrics = metrics)

In [ ]:
# convert numerical data into categorical data
y_sentiment = to_categorical(data_bank_final['label_numerical'])
y_aspect    = to_categorical(data_bank_final['aspect_numerical'])

In [ ]:
# tokenize sentence 
x = tokenizer(
    text = data_bank_final['sentence'].to_list(),
    add_special_tokens = True,
    max_length = MAX_LENGTH,
    truncation = True,
    padding = True, 
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True
)

In [ ]:
history_bank = model_bert_bank.fit(
    x = {'input_ids': x['input_ids'], 'attention_mask' : x['attention_mask']},
    y = {"sentiment" : y_sentiment, "aspect" : y_aspect},
    validation_split = 0.2,
    batch_size = 8,
    epochs = 10,
    callbacks = [OwnCallbacks()]
)

Epoch 1/10
2720/2720 [==============================] - 1527s 561ms/step - loss: 0.7851 - Aspect_loss: 0.5925 - Sentiment_loss: 0.1927 - Aspect_accuracy: 0.7715 - Sentiment_accuracy: 0.9412 - val_loss: 0.5900 - val_Aspect_loss: 0.4420 - val_Sentiment_loss: 0.1480 - val_Aspect_accuracy: 0.8039 - val_Sentiment_accuracy: 0.9520
Epoch 2/10
2720/2720 [==============================] - 1484s 546ms/step - loss: 0.5847 - Aspect_loss: 0.4346 - Sentiment_loss: 0.1502 - Aspect_accuracy: 0.8143 - Sentiment_accuracy: 0.9489 - val_loss: 0.5747 - val_Aspect_loss: 0.4350 - val_Sentiment_loss: 0.1397 - val_Aspect_accuracy: 0.7994 - val_Sentiment_accuracy: 0.9564


In [ ]:
model_bert_bank.save("model_bert_bank.h5")

In [ ]:
metrics_training_bank = pd.DataFrame(history_bank.history)
metrics_training_bank

,loss,Aspect_loss,Sentiment_loss,Aspect_accuracy,Sentiment_accuracy,val_loss,val_Aspect_loss,val_Sentiment_loss,val_Aspect_accuracy,val_Sentiment_accuracy
0,0.785136,0.592467,0.192669,0.771507,0.941222,0.590002,0.441968,0.148035,0.803860,0.952022
1,0.584720,0.434560,0.150160,0.814292,0.948897,0.574667,0.434996,0.139671,0.799449,0.956434


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# convert numerical data into categorical data
y_sentiment_test = to_categorical(data_bank_final_test['label_numerical'])
y_aspect_test    = to_categorical(data_bank_final_test['aspect_numerical'])

In [ ]:
# tokenize sentence 
x_test = tokenizer(
    text = data_bank_final_test['sentence'].to_list(),
    add_special_tokens = True,
    max_length = MAX_LENGTH,
    truncation = True,
    padding = True, 
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True
)

In [ ]:
model_bert_bank.evaluate( 
    x = {'input_ids' : x_test['input_ids'], 'attention_mask' : x_test['attention_mask']},
    y = {'sentiment': y_sentiment_test, 'aspect': y_aspect_test},
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


213/213 [==============================] - 57s 267ms/step - loss: 0.5908 - Aspect_loss: 0.4430 - Sentiment_loss: 0.1478 - Aspect_accuracy: 0.7932 - Sentiment_accuracy: 0.9526


[0.5908228754997253,
 0.44303932785987854,
 0.1477832794189453,
 0.7932353019714355,
 0.9526470303535461]

In [ ]:
predicted = model_bert_bank.predict(x = {'input_ids' : x_test['input_ids'], 'attention_mask' : x_test['attention_mask']})

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


In [ ]:
# sentiment
print(classification_report( 
    y_sentiment_test.argmax(axis =  1),
    predicted['sentiment'].argmax(axis = 1)
))

              precision    recall  f1-score   support

           0       0.67      0.13      0.22       138
           1       0.96      1.00      0.98      6394
           2       0.67      0.36      0.47       268

    accuracy                           0.95      6800
   macro avg       0.76      0.49      0.55      6800
weighted avg       0.94      0.95      0.94      6800



In [ ]:
# aspect
print(classification_report( 
    y_aspect_test.argmax(axis =  1),
    predicted['aspect'].argmax(axis = 1)
))

              precision    recall  f1-score   support

           0       0.80      0.89      0.84       680
           1       0.87      0.72      0.79       680
           2       0.82      0.68      0.74       680
           3       0.86      0.91      0.88       680
           4       0.65      0.70      0.67       680
           5       0.86      0.72      0.78       680
           6       0.74      0.76      0.75       680
           7       0.77      0.76      0.76       680
           8       0.72      0.84      0.78       680
           9       0.88      0.96      0.92       680

    accuracy                           0.79      6800
   macro avg       0.80      0.79      0.79      6800
weighted avg       0.80      0.79      0.79      6800



## 2.2 Food Dataset

In [ ]:
# split data into train and test
data_pangan_final, data_pangan_final_test = train_test_split(data_pangan_final, test_size = 0.2, stratify = data_pangan_final["aspect_numerical"])

# check dimension of data after splitted
print(data_pangan_final.shape, data_pangan_final_test.shape)

(28160, 5) (7040, 5)


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = MODEL_NAME, config = config)
model = TFBertModel.from_pretrained(MODEL_NAME, config)

Some layers from the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# inherit model
model_bert_pangan = build_model(model, data_pangan_final, "label_numerical", "aspect_numerical")

In [ ]:
# overview model architecture
model_bert_pangan.summary()

Model: "BERT_Bank_Multiclass"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 64)]         0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 64)]         0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  110617344   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]']              
                                tentions(last_hidde                                               
                                n_state=(None, 64,                             

In [ ]:
# compile model
model_bert_pangan.compile(optimizer = optimizer, loss = loss, metrics = metrics)

In [ ]:
# convert numerical data into categorical data
y_sentiment = to_categorical(data_pangan_final['label_numerical'])
y_aspect    = to_categorical(data_pangan_final['aspect_numerical'])

In [ ]:
# tokenize sentence 
x = tokenizer(
    text = data_pangan_final['sentence'].to_list(),
    add_special_tokens = True,
    max_length = MAX_LENGTH,
    truncation = True,
    padding = True, 
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True
)

In [ ]:
history_pangan = model_bert_pangan.fit(
    x = {'input_ids': x['input_ids'], 'attention_mask' : x['attention_mask']},
    y = {"sentiment" : y_sentiment, "aspect" : y_aspect},
    validation_split = 0.2,
    batch_size = 8,
    epochs = 10,
    callbacks = [OwnCallbacks()]
)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


2816/2816 [==============================] - 1606s 570ms/step - loss: 0.8914 - Aspect_loss: 0.6897 - Sentiment_loss: 0.2017 - Aspect_accuracy: 0.7402 - Sentiment_accuracy: 0.9393 - val_loss: 0.6974 - val_Aspect_loss: 0.5334 - val_Sentiment_loss: 0.1640 - val_Aspect_accuracy: 0.7683 - val_Sentiment_accuracy: 0.9466
Epoch 2/10
2816/2816 [==============================] - 1587s 564ms/step - loss: 0.6574 - Aspect_loss: 0.4990 - Sentiment_loss: 0.1585 - Aspect_accuracy: 0.7886 - Sentiment_accuracy: 0.9479 - val_loss: 0.6635 - val_Aspect_loss: 0.5086 - val_Sentiment_loss: 0.1548 - val_Aspect_accuracy: 0.7715 - val_Sentiment_accuracy: 0.9492
Epoch 3/10
2816/2816 [==============================] - 1591s 565ms/step - loss: 0.6194 - Aspect_loss: 0.4733 - Sentiment_loss: 0.1460 - Aspect_accuracy: 0.7930 - Sentiment_accuracy: 0.9515 - val_loss: 0.6548 - val_Aspect_loss: 0.5038 - val_Sentiment_loss: 0.1510 - val_Aspect_accuracy: 0.7701 - val_Sentiment_accuracy: 0.9510
Epoch 4/10
2816/2816 [========

In [ ]:
model_bert_pangan.save("model_bert_pangan.h5")

In [ ]:
metrics_training_pangan = pd.DataFrame(history_pangan.history)
metrics_training_pangan

,loss,Aspect_loss,Sentiment_loss,Aspect_accuracy,Sentiment_accuracy,val_loss,val_Aspect_loss,val_Sentiment_loss,val_Aspect_accuracy,val_Sentiment_accuracy
0,0.891395,0.689695,0.201700,0.740234,0.939320,0.697365,0.533381,0.163983,0.768288,0.946555
1,0.657441,0.498964,0.158478,0.788574,0.947931,0.663479,0.508649,0.154830,0.771484,0.949219
2,0.619362,0.473329,0.146034,0.792969,0.951527,0.654789,0.503762,0.151027,0.770064,0.950994
3,0.597868,0.458887,0.138982,0.798162,0.953835,0.654738,0.505162,0.149576,0.768643,0.951172
4,0.585031,0.450460,0.134571,0.801314,0.955922,0.650381,0.503539,0.146843,0.765803,0.951172


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# convert numerical data into categorical data
y_sentiment_test = to_categorical(data_pangan_final_test['label_numerical'])
y_aspect_test    = to_categorical(data_pangan_final_test['aspect_numerical'])

In [ ]:
# tokenize sentence 
x_test = tokenizer(
    text = data_pangan_final_test['sentence'].to_list(),
    add_special_tokens = True,
    max_length = MAX_LENGTH,
    truncation = True,
    padding = True, 
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True
)

In [ ]:
model_bert_pangan.evaluate( 
    x = {'input_ids' : x_test['input_ids'], 'attention_mask' : x_test['attention_mask']},
    y = {'sentiment': y_sentiment_test, 'aspect': y_aspect_test},
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


220/220 [==============================] - 45s 202ms/step - loss: 0.6709 - Aspect_loss: 0.5046 - Sentiment_loss: 0.1663 - Aspect_accuracy: 0.7626 - Sentiment_accuracy: 0.9453


[0.6708971261978149,
 0.5045595765113831,
 0.16633795201778412,
 0.7626420259475708,
 0.9453125]

In [ ]:
predicted = model_bert_pangan.predict(x = {'input_ids' : x_test['input_ids'], 'attention_mask' : x_test['attention_mask']})

In [ ]:
# sentiment
print(classification_report( 
    y_sentiment_test.argmax(axis =  1),
    predicted['sentiment'].argmax(axis = 1)
))

              precision    recall  f1-score   support

           0       0.33      0.03      0.06       126
           1       0.96      0.99      0.97      6579
           2       0.59      0.43      0.50       335

    accuracy                           0.95      7040
   macro avg       0.63      0.48      0.51      7040
weighted avg       0.93      0.95      0.93      7040



In [ ]:
# aspect
print(classification_report( 
    y_aspect_test.argmax(axis =  1),
    predicted['aspect'].argmax(axis = 1)
))

              precision    recall  f1-score   support

           0       0.86      0.59      0.70       704
           1       0.65      0.69      0.67       704
           2       0.73      0.85      0.79       704
           3       0.82      0.81      0.81       704
           4       0.78      0.73      0.75       704
           5       0.83      0.75      0.79       704
           6       0.78      0.85      0.81       704
           7       0.77      0.88      0.82       704
           8       0.72      0.71      0.71       704
           9       0.75      0.76      0.76       704

    accuracy                           0.76      7040
   macro avg       0.77      0.76      0.76      7040
weighted avg       0.77      0.76      0.76      7040



## 2.3 Goverment Dataset

In [17]:
# split data into train and test
data_gove_final, data_gove_final_test = train_test_split(data_gove_final, test_size = 0.2, stratify = data_gove_final["aspect_numerical"])

# check dimension of data after splitted
print(data_gove_final.shape, data_gove_final_test.shape)

(19200, 5) (4800, 5)


In [18]:
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = MODEL_NAME, config = config)
model = TFBertModel.from_pretrained(MODEL_NAME, config)

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545M [00:00<?, ?B/s]

Some layers from the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [19]:
# inherit model
model_bert_gove = build_model(model, data_gove_final, "label_numerical", "aspect_numerical")

In [20]:
# overview model architecture
model_bert_gove.summary()

Model: "BERT_Bank_Multiclass"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask (InputLayer)    [(None, 64)]         0           []                               
                                                                                                  
 input_ids (InputLayer)         [(None, 64)]         0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  110617344   ['attention_mask[0][0]',         
                                thPoolingAndCrossAt               'input_ids[0][0]']              
                                tentions(last_hidde                                               
                                n_state=(None, 64,                             

In [21]:
# compile model
model_bert_gove.compile(optimizer = optimizer, loss = loss, metrics = metrics)

In [22]:
# convert numerical data into categorical data
y_sentiment = to_categorical(data_gove_final['label_numerical'])
y_aspect    = to_categorical(data_gove_final['aspect_numerical'])

In [23]:
# tokenize sentence 
x = tokenizer(
    text = data_gove_final['sentence'].to_list(),
    add_special_tokens = True,
    max_length = MAX_LENGTH,
    truncation = True,
    padding = True, 
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True
)

In [24]:
history_gove = model_bert_gove.fit(
    x = {'input_ids': x['input_ids'], 'attention_mask' : x['attention_mask']},
    y = {"sentiment" : y_sentiment, "aspect" : y_aspect},
    validation_split = 0.2,
    batch_size = 8,
    epochs = 10,
    callbacks = [OwnCallbacks()]
)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


1920/1920 [==============================] - 1139s 593ms/step - loss: 0.6685 - Aspect_loss: 0.4882 - Sentiment_loss: 0.1802 - Aspect_accuracy: 0.8199 - Sentiment_accuracy: 0.9462 - val_loss: 0.5374 - val_Aspect_loss: 0.3807 - val_Sentiment_loss: 0.1568 - val_Aspect_accuracy: 0.8341 - val_Sentiment_accuracy: 0.9490


In [25]:
model_bert_gove.save("model_bert_gove.h5")

In [26]:
metrics_training_gove = pd.DataFrame(history_gove.history)
metrics_training_gove

,loss,Aspect_loss,Sentiment_loss,Aspect_accuracy,Sentiment_accuracy,val_loss,val_Aspect_loss,val_Sentiment_loss,val_Aspect_accuracy,val_Sentiment_accuracy
0,0.66846,0.488221,0.180239,0.819922,0.946224,0.537432,0.380662,0.15677,0.834115,0.948958


In [27]:
from sklearn.metrics import classification_report

In [28]:
# convert numerical data into categorical data
y_sentiment_test = to_categorical(data_gove_final_test['label_numerical'])
y_aspect_test    = to_categorical(data_gove_final_test['aspect_numerical'])

In [29]:
# tokenize sentence 
x_test = tokenizer(
    text = data_gove_final_test['sentence'].to_list(),
    add_special_tokens = True,
    max_length = MAX_LENGTH,
    truncation = True,
    padding = True, 
    return_tensors = 'tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True
)

In [30]:
model_bert_gove.evaluate( 
    x = {'input_ids' : x_test['input_ids'], 'attention_mask' : x_test['attention_mask']},
    y = {'sentiment': y_sentiment_test, 'aspect': y_aspect_test},
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


150/150 [==============================] - 36s 243ms/step - loss: 0.5240 - Aspect_loss: 0.3770 - Sentiment_loss: 0.1470 - Aspect_accuracy: 0.8329 - Sentiment_accuracy: 0.9540


[0.5239651799201965,
 0.37700557708740234,
 0.1469595730304718,
 0.8329166769981384,
 0.9539583325386047]

In [31]:
predicted = model_bert_gove.predict(x = {'input_ids' : x_test['input_ids'], 'attention_mask' : x_test['attention_mask']})

In [32]:
# sentiment
print(classification_report( 
    y_sentiment_test.argmax(axis =  1),
    predicted['sentiment'].argmax(axis = 1)
))

              precision    recall  f1-score   support

           0       0.33      0.01      0.02        79
           1       0.96      0.99      0.98      4551
           2       0.57      0.38      0.46       170

    accuracy                           0.95      4800
   macro avg       0.62      0.46      0.49      4800
weighted avg       0.94      0.95      0.94      4800



In [33]:
# aspect
print(classification_report( 
    y_aspect_test.argmax(axis =  1),
    predicted['aspect'].argmax(axis = 1)
))

              precision    recall  f1-score   support

           0       0.89      0.63      0.74       480
           1       0.76      0.77      0.76       480
           2       0.84      0.79      0.82       480
           3       0.86      0.93      0.89       480
           4       0.80      0.89      0.84       480
           5       0.82      0.93      0.87       480
           6       0.81      0.69      0.74       480
           7       0.81      0.75      0.78       480
           8       0.80      0.97      0.88       480
           9       0.96      0.97      0.96       480

    accuracy                           0.83      4800
   macro avg       0.83      0.83      0.83      4800
weighted avg       0.83      0.83      0.83      4800

